<a href="https://colab.research.google.com/github/maximecharriere/CryptoWallet/blob/main/CryptoWallet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Import libraries


In [1]:
import os
from CryptoWallet.Loader import BinanceLoader, SwissborgLoader, KucoinLoader, BybitLoader, ManualTransactionsLoader, CoinbaseLoader, LedgerLoader
from CryptoWallet.Wallet import Wallet
from CryptoWallet.Settings import Settings

import pandas as pd

settings = Settings.load()

# Open database


In [ ]:
wallet = Wallet(apiKey=settings.cryptocompare_api_key, databaseFilename=settings.database_filepath)

## Import Binance transactions

### ~~Payment History~~
- (déjà dans Transaction History)
### Convert History
- (??. Pas encore de transactions)
### Fiat Orders
- (??. Pas encore de transactions)
### Earn History - Simple Earn - Flexible
- ~~earn-flexible-subscription~~ (déjà dans Transaction History)  
- ~~earn-flexible-redemption~~ (déjà dans Transaction History)  
- **earn-flexible-rewards** -> Enlever les "Bonus Tiered APR Rewards" (déjà dans Transaction History), ajouter "Real-time APR Rewards" a la Transaction History  
> Transformer `"2024-12-30","BNB","6.4E-7","Real-time APR Rewards"`  
> en `"141795728","2024-12-30 00:00:00","Spot","Simple Earn Flexible Interest","BNB","6.4E-7","Manually added from export : earn-flexible-rewards"`  

### Earn History - Simple Earn - Locked / Flexible Trial Fund / Collateral / Locked Savings / Activities
- (??. Pas encore de transactions)
### Earn History - Staking - ETH
- **earn-eth-staking**  
- **earn-eth-redemption**  
- ?? earn-eth-distribution (??. Pas encore de transactions)
- ?? earn-eth-wrap (??. Pas encore de transactions)
- ?? earn-eth-unwrap (??. Pas encore de transactions)
### Earn History - Staking - SOL
- **earn-sol-staking**  
- **earn-sol-redemption**  
- ~~earn-sol-rewards~~ (n'est pas représentatif d'un réel changement dans les comptes, étant donné que c'est la valeur du BNSOL qui monte, et que aucun SOL n'est distribué)
- ?? earn-sol-claim (l'export ne fonctionne pas)  
- ?? earn-sol-distribution (l'export ne fonctionne pas)  
### Earn History - Staking - On-chain / DeFi / DOT / NFT
- (??. Pas encore de transactions)
### ~~Earn History - Lauchpool~~
- (déjà dans Transaction History)
### Earn History - Dual / Auto / Range / Liquid / Smart
- (??. Pas encore de transactions)
### P2P Oder
- (??. Pas encore de transactions)
### Futures Order
- (??. Pas encore de transactions)
### ~~Spot Order~~
- (déjà dans Transaction History)


In [ ]:
transactions_path = os.path.join(settings.exported_transactions_dirpath, "Binance")

for filename in os.listdir(transactions_path):
    # Create the full file path
    filepath = os.path.join(transactions_path, filename)

    # Check if it's a file and not a directory
    if os.path.isfile(filepath):
        # Call your processing function
        wallet.addTransactions(BinanceLoader.load(filepath))

wallet.printFirstLastTransactionDatetime()
# wallet.save()

## Import Ledger transactions
- Ledger Live > Accounts > ... > Export

Note:  
- CORE tokens must be added manually (CORE network)
- RWA tokens must be added manually (BASE network)

In [ ]:
transactions_path = os.path.join(settings.exported_transactions_dirpath, "Ledger")

for filename in os.listdir(transactions_path):
    # Create the full file path
    filepath = os.path.join(transactions_path, filename)

    # Check if it's a file and not a directory
    if os.path.isfile(filepath):
        # Call your processing function
        wallet.addTransactions(LedgerLoader.load(filepath))

wallet.printFirstLastTransactionDatetime()
# wallet.save()

## Import Swissborg transactions

1. Go in Swissborg app
2. Profil > Relevé de compte

In [ ]:
transactions_path = os.path.join(settings.exported_transactions_dirpath, "SwissBorg")

for filename in os.listdir(transactions_path):
    # Create the full file path
    filepath = os.path.join(transactions_path, filename)

    # Check if it's a file and not a directory
    if os.path.isfile(filepath):
        # Call your processing function
        wallet.addTransactions(SwissborgLoader.load(filepath))

wallet.printFirstLastTransactionDatetime()
# wallet.save()

## Import Kucoin transactions
1. Go to https://www.kucoin.com/order/trade/history
2. Export all account history


In [ ]:
transactions_path = os.path.join(settings.exported_transactions_dirpath, "Kucoin")

for folder_name in os.listdir(transactions_path):
    # Create the full file path
    subfolder_path = os.path.join(transactions_path, folder_name)

    # Call your processing function
    wallet.addTransactions(KucoinLoader.load(subfolder_path))

wallet.printFirstLastTransactionDatetime()
# wallet.save()

## Import Bybit transactions

1. Go to https://www.bybit.com/user/assets/exportStatment
2. Export "Account Change Details" from "Export Account Statement"

In [ ]:
transactions_path = os.path.join(settings.exported_transactions_dirpath, "ByBit")

for folder_name in os.listdir(transactions_path):
    # Create the full file path
    subfolder_path = os.path.join(transactions_path, folder_name)

    # Call your processing function
    wallet.addTransactions(BybitLoader.load(subfolder_path))
    
wallet.printFirstLastTransactionDatetime()
# wallet.save()

# Import Coinbase transactions

1. Go to https://accounts.coinbase.com/statements
2. Export all as CSV

## Import Manual Transactions files

In [ ]:
transactions_path = os.path.join(settings.exported_transactions_dirpath, "Manual")

for filename in os.listdir(transactions_path):
    # Create the full file path
    filepath = os.path.join(transactions_path, filename)

    # Check if it's a file and not a directory
    if os.path.isfile(filepath):
        # Call your processing function
        wallet.addTransactions(ManualTransactionsLoader.load(filepath))

# wallet.save()

## Summary

In [ ]:
wallet.getSummary()

## Export Excel


In [5]:
stats_filepath = os.path.join(settings.output_dirpath, "stats.xlsx")

if settings.output_dirpath:  # Only create directories if there's actually a path component
    os.makedirs(settings.output_dirpath, exist_ok=True)

with pd.ExcelWriter(stats_filepath, engine='openpyxl') as writer:
    wallet.getSummary().to_excel(writer, sheet_name='Summary')
    wallet.getCoinsStats().to_excel(writer, sheet_name='Coins Stats')
    wallet.getWalletsBalance().to_excel(writer, sheet_name='Wallets Balance')
    wallet.getTransactions(remove_datetime_timezone=True).to_excel(writer, sheet_name='Transactions', index=False)


# Export for TradingView


In [ ]:
TradingView_filepath = os.path.join(settings.output_dirpath, "TradingView_PineData.txt")

if settings.output_dirpath:  # Only create directories if there's actually a path component
    os.makedirs(settings.output_dirpath, exist_ok=True)
    
wallet.exportTradingView(TradingView_filepath)

# Save transactions


In [ ]:
wallet.save()